In [ ]:
%cd /with_history/dpo

In [ ]:
# DMPO & SDPO

In [ ]:
category='Video_Games'
start_year=2015
end_year=2018
category_dict = {"CDs_and_Vinyl": "musics", "Video_Games": "video games", 'Movies_and_TV': 'movies and TV'}
item_category = category_dict[category]

In [ ]:
train_file=f"../train/{category}_5_{start_year}-10-{end_year}-11.csv"
eval_file=f"../valid/{category}_5_{start_year}-10-{end_year}-11.csv"
info_file=f"../info/{category}_5_{start_year}-10-{end_year}-11.txt"
train_input = []
eval_input = []

In [ ]:
def get_history(row, category):
    row['history_item_title'] = eval(row['history_item_title'])
    L = len(row['history_item_title']) 
    history = ""
    for i in range(L):
        if i == 0:
            history += row['history_item_title'][i]
        else:
            history += ", " + row['history_item_title'][i]     
    target_item = str(row['item_title'])
    target_item = target_item
    target_item_id = row["item_id"]
    last_history_item_id = eval(row["history_item_id"])[-1]
    return {"input": f"The user has enjoyed the following {category}s before: {history}",
            "output": target_item + '\n',
            "dedup": target_item_id == last_history_item_id}

In [ ]:
all_item = []
with open(info_file, 'r') as f:
    info = f.readlines()
    all_item = [_.split('\t')[0].strip(' ') + "\n" for _ in info]
print(len(all_item))

In [ ]:
import random
import pandas as pd
from tqdm import tqdm
def generate_prompt(data_point, category):
    return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. 

### Instruction:
Given a list of {category} the user recently enjoy, please write a new {category} that the user may bought

### User Input: 
{data_point["input"]}

### Response: 
"""
train_data = pd.read_csv(train_file)
eval_data = pd.read_csv(eval_file)
for i in tqdm(range(len(train_data))):
    history = get_history(train_data.iloc[i], item_category)
    other_item = list(filter(lambda x: x != history['output'], all_item))
    rejecteds = random.sample(other_item, 3)
    train_input.append({'prompt':generate_prompt(history, item_category), 'chosen':history['output'], 'rejected1':rejecteds[0], 'rejected2':rejecteds[1], 'rejected3':rejecteds[2]})
for i in tqdm(range(len(eval_data))):
    history = get_history(eval_data.iloc[i], item_category)
    other_item = list(filter(lambda x: x != history['output'], all_item))
    rejecteds = random.sample(other_item, 3)
    eval_input.append({'prompt':generate_prompt(history, item_category), 'chosen':history['output'], 'rejected1':rejecteds[0], 'rejected2':rejecteds[1], 'rejected3':rejecteds[2]})

In [ ]:
import json
with open(f'./data/{category}/{category}_dpo_train_dataset.json', 'w') as f:
    json.dump(train_input, f, indent=4)
with open(f'./data/{category}/{category}_dpo_eval_dataset.json', 'w') as f:
    json.dump(eval_input, f, indent=4)

In [ ]:
# RoseDPO

In [ ]:
category='Video_Games'
start_year=2015
end_year=2018
category_dict = {"CDs_and_Vinyl": "musics", "Video_Games": "video games", 'Movies_and_TV': 'movies and TV'}
item_category = category_dict[category]

In [ ]:
train_file=f"../train/{category}_5_{start_year}-10-{end_year}-11.csv"
eval_file=f"../valid/{category}_5_{start_year}-10-{end_year}-11.csv"
info_file=f"../info/{category}_5_{start_year}-10-{end_year}-11.txt"
title_interaction=f"../info/{category}_title_interaction_5_{start_year}-10-{end_year}-11.json"
train_input = []
eval_input = []

In [ ]:
def get_history(row, category):
    row['history_item_title'] = eval(row['history_item_title'])
    L = len(row['history_item_title']) 
    history = ""
    for i in range(L):
        if i == 0:
            history += row['history_item_title'][i]
        else:
            history += ", " + row['history_item_title'][i]     
    target_item = str(row['item_title'])
    target_item = target_item
    target_item_id = row["item_id"]
    last_history_item_id = eval(row["history_item_id"])[-1]
    return {"input": f"The user has enjoyed the following {category}s before: {history}",
            "output": target_item + '\n',
            "dedup": target_item_id == last_history_item_id}

In [ ]:
def get_prob(row, title2inter):
    row['history_item_title'] = eval(row['history_item_title'])
    L = len(row['history_item_title']) 
    for i in range(L):
        if row['history_item_title'][i] not in title2inter:
            title2inter[row['history_item_title'][i]] = 0
        title2inter[row['history_item_title'][i]] += 1
    return title2inter

In [ ]:
all_item = []
with open(info_file, 'r') as f:
    info = f.readlines()
    all_item = [_.split('\t')[0].strip(' ') + "\n" for _ in info]
print(len(all_item))

In [ ]:
import random
import pandas as pd
from tqdm import tqdm
def generate_prompt(data_point, category):
    return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. 

### Instruction:
Given a list of {category} the user recently enjoy, please write a new {category} that the user may bought

### User Input: 
{data_point["input"]}

### Response: 
"""
train_data = pd.read_csv(train_file)
eval_data = pd.read_csv(eval_file)
with open(title_interaction, "r") as input_file:
    title_inter = json.load(input_file)
for i in tqdm(range(len(train_data))):
    history = get_history(train_data.iloc[i], item_category)
    new_list = list(set(all_item.copy()))
    new_list.remove(history['output'])
    t2i = title_inter.copy()
    if history['output'].strip('\n') in t2i:
        del t2i[history['output'].strip('\n')] 
    total = sum(t2i.values())
    t2i = {k+"\n": v / total for k, v in t2i.items()}
    weights = [t2i.get(item, 0) for item in new_list]
    rejecteds = random.choices(new_list, weights=weights, k=1)[0]
    train_input.append({'prompt':generate_prompt(history, item_category), 'chosen':history['output'], 'rejected':rejecteds})
for i in tqdm(range(len(eval_data))):
    history = get_history(eval_data.iloc[i], item_category)
    new_list = list(set(all_item.copy()))
    new_list.remove(history['output'])
    t2i = title_inter.copy()
    if history['output'].strip('\n') in t2i:
        del t2i[history['output'].strip('\n')] 
    total = sum(t2i.values())
    t2i = {k+"\n": v / total for k, v in t2i.items()}
    weights = [t2i.get(item, 0) for item in new_list]
    rejecteds = random.choices(new_list, weights=weights, k=1)[0]
    eval_input.append({'prompt':generate_prompt(history, item_category), 'chosen':history['output'], 'rejected':rejecteds})

In [ ]:
import json
with open(f'./data/{category}/{category}_rosedpo_train_dataset.json', 'w') as f:
    json.dump(train_input, f, indent=4)
with open(f'./data/{category}/{category}_rosedpo_eval_dataset.json', 'w') as f:
    json.dump(eval_input, f, indent=4)